In [9]:
import numpy as np
import matplotlib.pyplot as plt
import math

## load_data

In [46]:
import struct
import matplotlib.pyplot as plt

# 训练集文件
train_images_idx3_ubyte_file = '../data/common/train-images-idx3-ubyte'
# 训练集标签文件
train_labels_idx1_ubyte_file = '../data/common/train-labels-idx1-ubyte'

# 测试集文件
test_images_idx3_ubyte_file = '../data/common/t10k-images-idx3-ubyte'
# 测试集标签文件
test_labels_idx1_ubyte_file = '../data/common/t10k-labels-idx1-ubyte'
def decode_idx3_ubyte(idx3_ubyte_file):
    """
    解析idx3文件的通用函数
    :param idx3_ubyte_file: idx3文件路径
    :return: 数据集
    """
    # 读取二进制数据
    bin_data = open(idx3_ubyte_file, 'rb').read()

    # 解析文件头信息，依次为魔数、图片数量、每张图片高、每张图片宽
    offset = 0
    fmt_header = '>iiii'
    magic_number, num_images, num_rows, num_cols = struct.unpack_from(fmt_header, bin_data, offset)
    print('魔数:%d, 图片数量: %d张, 图片大小: %d*%d' % (magic_number, num_images, num_rows, num_cols))

    # 解析数据集
    image_size = num_rows * num_cols
    offset += struct.calcsize(fmt_header)
    fmt_image = '>' + str(image_size) + 'B'
    images = np.empty((num_images, num_rows, num_cols))
    for i in range(num_images):
        if (i + 1) % 10000 == 0:
            print('已解析 %d' % (i + 1) + '张')
        images[i] = np.array(struct.unpack_from(fmt_image, bin_data, offset)).reshape((num_rows, num_cols))
        offset += struct.calcsize(fmt_image)
    return images


def decode_idx1_ubyte(idx1_ubyte_file):
    """
    解析idx1文件的通用函数
    :param idx1_ubyte_file: idx1文件路径
    :return: 数据集
    """
    # 读取二进制数据
    bin_data = open(idx1_ubyte_file, 'rb').read()

    # 解析文件头信息，依次为魔数和标签数
    offset = 0
    fmt_header = '>ii'
    magic_number, num_images = struct.unpack_from(fmt_header, bin_data, offset)
    print('魔数:%d, 图片数量: %d张' % (magic_number, num_images))

    # 解析数据集
    offset += struct.calcsize(fmt_header)
    fmt_image = '>B'
    labels = np.empty(num_images)
    for i in range(num_images):
        if (i + 1) % 10000 == 0:
            print('已解析 %d' % (i + 1) + '张')
        labels[i] = struct.unpack_from(fmt_image, bin_data, offset)[0]
        offset += struct.calcsize(fmt_image)
    return labels
train_image=decode_idx3_ubyte(train_images_idx3_ubyte_file)
test_image=decode_idx3_ubyte(test_images_idx3_ubyte_file)
train_label=decode_idx1_ubyte(train_labels_idx1_ubyte_file)
test_label=decode_idx1_ubyte(test_labels_idx1_ubyte_file)

魔数:2051, 图片数量: 60000张, 图片大小: 28*28
已解析 10000张
已解析 20000张
已解析 30000张
已解析 40000张
已解析 50000张
已解析 60000张
魔数:2051, 图片数量: 10000张, 图片大小: 28*28
已解析 10000张
魔数:2049, 图片数量: 60000张
已解析 10000张
已解析 20000张
已解析 30000张
已解析 40000张
已解析 50000张
已解析 60000张
魔数:2049, 图片数量: 10000张
已解析 10000张


In [47]:
def get_onezero_size(labels):
    size=labels.shape[0]
    j=0
    for i in range(size):
        if (labels[i]==0)or(labels[i]==1):
            j+=1
    return j
train_num=get_onezero_size(train_label)
test_num=get_onezero_size(train_label)
def get_onezeros_sample(num,images,labels):
    images1=np.empty((num,28,28))
    labels1=np.empty(num)
    size=labels.shape[0]
    j=0
    for i in range(size):
        if (labels[i]==0)or(labels[i]==1):
            images1[j]=images[i]
            labels1[j]=labels[i]
            j+=1
    return images1,labels1
train_images,train_labels=get_onezeros_sample(train_num,train_image,train_label)
test_images,test_labels=get_onezeros_sample(test_num,test_image,test_label)

## ex_logi_reg

In [4]:
#functions [f,g] = logistic_regression_vec(theta, X,y)
#   %
#   % Arguments:
#   %   theta - A column vector containing the parameter values to optimize.
#   %   X - The examples stored in a matrix.  
#   %       X(i,j) is the i'th coordinate of the j'th example.
#   %   y - The label for each example.  y(j) is the j'th example's label.
#   %
def log_reg(theta,X,y):
    m=X.shape[1];
    n=X.shape[0];
  #   % initialize objective value and gradient.
    f = 0;
    g=np.zeros(theta.shape[1])
    temp=np.dot(theta,X)
    print("temp,",temp)
    h_temp=1/(1+np.exp(-temp))
    print("h_temp",h_temp)
    cost=y*np.log(h_temp)+(1-y)*np.log(1-h_temp)
    f=-np.sum(cost)
    print("cost:",f)
    for j in range(0,n): 
        
        g[j]=np.sum((h_temp-y)*X[j,:])
    return f,g

#TODO:  Compute the logistic regression objective function and gradient 
#   %        using vectorized code.  (It will be just a few lines of code!)
#   %        Store the objective function value in 'f', and the gradient in 'g'.
#   %
# %%% YOUR CODE HERE %%%

In [32]:
train_X=np.empty((784,train_num))
test_X=np.empty((784,test_num))
for i in range(10):
    train_X[:,i]=train_images[i].reshape(784)
print(train_X.shape)

(784, 12665)


In [48]:
# % Load the MNIST data for this exercise.
# % train.X and test.X will contain the training and testing images.
# %   Each matrix has size [n,m] where:
# %      m is the number of examples.
# %      n is the number of pixels in each image.
# % train.y and test.y will contain the corresponding labels (0 or 1).
train_X=np.empty((784,train_num))
test_X=np.empty((784,test_num))
for i in range(train_num):
    train_X[:,i]=train_images[i].reshape(784)
for i in range(test_num):
    test_X[:,i]=test_images[i].reshape(784)
cols1=train_X.shape[1]
append1=np.ones(shape=(1,cols1))
cols2=test_X.shape[1]
append2=np.ones(shape=(1,cols2))
# % Include a row of 1s as an additional intercept feature.
# % Add row of 1s to the dataset to act as an intercept term.
train_X = np.insert(train_X,0,values=append1,axis=0)
test_X = np.insert(test_X,0,values=append2,axis=0)

#% Training set dimensions
m=train_X.shape[1]
n=train_X.shape[0]

#% Train logistic regression classifier using minFunc
#options = struct('MaxIter', 100);

#% First, we initialize theta to some small random values.
#theta = np.random.random(n).reshape(1,n);
theta = init_weights(n)

In [ ]:
train_y=train_labels
test_y=test_labels
def ex_logistic_reg(theta,X,y,lr,epoches):
    res_theta=theta
    costmin=1000000
    for i in range(epoches):
        f,g=log_reg(theta,X,y)
        print(f)
        print("the cost is %f",f)
        theta-=lr*g
        if costmin>f:
            costmin=f
            res_theta=theta
    return res_theta
lr=0.5
epoches=1000
costmin=1000   
res_theta=ex_logistic_reg(theta,train_X,train_y,lr,epoches)

In [50]:
def binary_classifier_accuracy(theta,X,y):
    res=np.dot(theta,X)
    h_res=1/(1+np.exp(-res))
    predicted_y=h_res>0.5
    lens=len(y)
    acu=np.sum(predicted_y==y)/lens
    return acu,predicted_y
accuracy ,predicted_y= binary_classifier_accuracy(res_theta,train_X,train_y);
print('Training accuracy: %2.1f%%\n', 100*accuracy);
print(predicted_y)

Training accuracy: %2.1f%%
 99.9289380182
[[False  True  True ...,  True False  True]]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


In [51]:
accuracy ,predicted_y= binary_classifier_accuracy(res_theta,test_X,test_y);
print('Training accuracy: %2.1f%%\n', 100*accuracy);

Training accuracy: %2.1f%%
 16.6837741808


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# % Load the MNIST data for this exercise.
# % train.X and test.X will contain the training and testing images.
# %   Each matrix has size [n,m] where:
# %      m is the number of examples.
# %      n is the number of pixels in each image.
# % train.y and test.y will contain the corresponding labels (0 or 1).
train_X=np.empty((784,train_num))
test_X=np.empty((784,test_num))
for i in range(train_num):
    train_X[:,i]=train_images[i].reshape(784)
for i in range(test_num):
    test_X[:,i]=test_images[i].reshape(784)
cols1=train_X.shape[1]
append1=np.ones(shape=(1,cols1))
cols2=test_X.shape[1]
append2=np.ones(shape=(1,cols2))
# % Include a row of 1s as an additional intercept feature.
# % Add row of 1s to the dataset to act as an intercept term.
train_X = np.insert(train_X,0,values=append1)
test_X = np.insert(test_X,0,values=append2)
train_y=train_labels
test_y=test_labels
#% Training set dimensions
m=train_X.shape[1]
n=train_X.shape[0]

#% Train logistic regression classifier using minFunc
#options = struct('MaxIter', 100);

#% First, we initialize theta to some small random values.
theta = np.random.randn(1,n)*0.01;

# % Call minFunc with the logistic_regression.m file as the objective function.
# %
# % TODO:  Implement batch logistic regression in the logistic_regression.m file!
# %
# tic;
# theta=minFunc(@logistic_regression, theta, options, train.X, train.y);
# fprintf('Optimization took %f seconds.\n', toc);

# % Now, call minFunc again with logistic_regression_vec.m as objective.
# %
# % TODO:  Implement batch logistic regression in logistic_regression_vec.m using
# % MATLAB's vectorization features to speed up your code.  Compare the running
# % time for your logistic_regression.m and logistic_regression_vec.m implementations.
# %
def init_weights(n_features):
    # 初始化参数
    # 参数范围(-1/sqrt(N), 1/sqrt(N))
    limit = np.sqrt(1/n_features)
    w = np.random.uniform(-limit, limit, (1,n_features))
    return w
def ex_logistic_reg(theta,X,y,lr,epoches):
    res_theta=theta
    for i in range(epoches):
        [f,g]=log_reg(theta,X,y)
        print("the cost is %f",f)
        theta-=lr*g
        if costmin>f:
            costmin=f
            res_theta=theta
    return res_theta
lr=0.05
epoches=1000
costmin=1000   
res_theta=ex_logistic_reg(theta,train_X,train_y,lr,epoches)
# % Uncomment the lines below to run your vectorized code.
# %theta = rand(n,1)*0.001;
# %tic;
# %theta=minFunc(@logistic_regression_vec, theta, options, train.X, train.y);
# %fprintf('Optimization took %f seconds.\n', toc);

# % Print out training accuracy.
def binary_classifier_accuracy(theta,X,y):
    res=np.dot(theta,X)
    h_res=1/(1+np.exp(-res))
    predicted_y=h_res>0.5
    lens=len(y)
    acu=np.sum(predicted_y==y)/lens
    return acu
accuracy = binary_classifier_accuracy(res_theta,train_X,train_y);
print('Training accuracy: %2.1f%%\n', 100*accuracy);

#% Print out accuracy on the test set.
accuracy = binary_classifier_accuracy(res_theta,test_X,test_y);
print('Test accuracy: %2.1f%%\n', 100*accuracy);


## ex_iris

In [19]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

data = datasets.load_iris()
X = data['data']
y = data['target']
# n:特征数，k:标签数
n = X.shape[1]
label = np.unique(y)
k = len(label)

In [32]:
data = datasets.load_iris()
X = data.data[data.target != 0]/np.linalg.norm(data.data[data.target != 0], axis=0)
y = data.target[data.target != 0]
y[y == 1] = 0
y[y == 2] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [33]:
def init_weights(n_features):
    # 初始化参数
    # 参数范围(-1/sqrt(N), 1/sqrt(N))
    limit = np.sqrt(1/n_features)
    w = np.random.uniform(-limit, limit, (1,n_features))
    return w


In [41]:
def ex_logistic_reg(theta,X,y,lr,epoches):
    res_theta=theta
    costmin=10000
    for i in range(epoches):
        [f,g]=log_reg(theta,X,y)
        print("the cost is %f"%f)
        theta-=lr*g
        if costmin>f:
            costmin=f
            res_theta=theta
    return res_theta

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.75, test_size=0.15, random_state=1)
X_train=X_train.T
X_train=np.insert(X_train, 0, 1, axis=0)  # 添加一行
X_test=X_test.T
X_test=np.insert(X_test, 0, 1, axis=0)  # 添加一行
print(X_train.shape,X_test.shape)

(5, 75) (5, 15)


In [38]:
print(X_train.shape)

(5, 75)


In [ ]:
m=X_train.shape[1]
n=X_train.shape[0]
theta = init_weights(n)
print(theta)
epoches=1000
res_theta=ex_logistic_reg(theta,X_train,Y_train,lr,epoches)

In [ ]:
accuracy = binary_classifier_accuracy(res_theta,X_train,Y_train);
print('Training accuracy: %2.1f%\n'%100*accuracy);

#% Print out accuracy on the test set.
accuracy = binary_classifier_accuracy(res_theta,X_test,Y_test);
print('Test accuracy: %2.1f%\n'%100*accuracy);

In [ ]:
def cost_func(theta,X,y):
    m=X.shape[1];
    n=X.shape[0];
  #   % initialize objective value and gradient.
    f = 0;
    g=np.zeros(theta.shape[1])
    temp=np.dot(theta,X)
    print("temp,",temp)
    h_temp=1/(1+np.exp(-temp))
    print("h_temp",h_temp)
    cost=y*np.log(h_temp)+(1-y)*np.log(1-h_temp)
    f=-np.sum(cost)
    return f
def gredient_check(f,g,theta,X,y):
    epsilon = 1e-4
    checked=[]
    for i in range(len(theta)):
        theta1=theta
        theta2=theta
        theta1[i]-=epsilon
        theta2[i]+=epsilon
        check=(cost_func(theta2,X,y)-cost_func(theta1,X,y))/2*epsilon
        checked.append(check)
        print("gredient of %d is"%i,g[i])
        print("checked_gredient of %d is"%i,check)